# WetGrass analyzed using Edward

In [1]:
import numpy as np
import tensorflow as tf

import edward as ed
import edward.models as edm
import edward.inferences as edi

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
import os
import sys
cur_dir_path = os.getcwd()
print(cur_dir_path)
os.chdir('../../')
qfog_path = os.getcwd()
print(qfog_path)
sys.path.insert(0,qfog_path)

/home/bob/Notebooks/Quantum/quantum-fog/jupyter-notebooks/inference_via_ext_software
/home/bob/Notebooks/Quantum/quantum-fog


In [3]:
# build BayesNet object bnet from bif file
import importlib
mm = importlib.import_module("jupyter-notebooks.inference_via_ext_software.ModelMaker")
from graphs.BayesNet import *
in_path = "examples_cbnets/WetGrass.bif"
bnet = BayesNet.read_bif(in_path, False)

In [4]:
# build no_placeholders=True model from bnet
file_prefix = "examples_cbnets/WetGrass_no_phs"
mm.ModelMaker.write_edward_model(file_prefix, bnet, vtx_to_data=None, no_placeholders=True)

.py file with model can be found here

<a href=../../examples_cbnets/WetGrass_no_phs_edward.py>
../../examples_cbnets/WetGrass_no_phs_edward.py</a>

In [5]:
# -i option allows it to access notebook's namespace
%run -i examples_cbnets/WetGrass_no_phs_edward.py

In [6]:
# data
Cloudy_data = tf.constant(0, dtype=tf.int32)
WetGrass_data = tf.constant(1, dtype=tf.int32)

In [13]:
# run
q_dict = {Rain: Rain_q, Sprinkler: Sprinkler_q}
data_dict = {Cloudy: Cloudy_data, WetGrass: WetGrass_data}
inf = edi.KLqp(q_dict, data=data_dict)
inf.run(n_samples = 5, n_iter=1000)

1000/1000 [100%] ██████████████████████████████ Elapsed: 2s | Loss: 1.014


In [14]:
# print final prob distributions

print('\nRain')
print(Rain.probs.eval())

print('\nSprinkler')
print(Sprinkler.probs.eval())


Rain
[ 0.40000001  0.60000002]

Sprinkler
[ 0.2         0.80000001]


## Exact results using junction tree algorithm

In [15]:
from inference.JoinTreeEngine import *
jtree_eng = JoinTreeEngine(bnet)

# introduce some evidence
bnet.get_node_named("Cloudy").active_states = [0]
bnet.get_node_named("WetGrass").active_states = [1]

#print node distributiona
node_list = jtree_eng.bnet_ord_nodes
jtree_pot_list = jtree_eng.get_unipot_list(node_list)
for k in range(len(node_list)):
    print(jtree_pot_list[k])
    print('')

['Cloudy']
[ 1.  0.]

['Rain']
[ 0.34839842  0.65160158]

['Sprinkler']
[ 0.13119789  0.86880211]

['WetGrass']
[ 0.  1.]

